In [20]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from gensim import corpora, models
import spacy
import matplotlib.pyplot as plt
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import os
from collections import Counter

target_column = 'rawContent'

# Download NLTK data
nltk.download('stopwords')

# Load data
df = pd.read_csv("your_data.csv")
texts = df[target_column].astype(str).tolist()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# Initialize spaCy
nlp = spacy.load('en_core_web_sm')
# Cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

cleaned_texts = [clean_text(text) for text in texts]

# Tokenization and Lemmatization
def tokenize_lemmatize(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stop_words]

tokenized_texts = [tokenize_lemmatize(text) for text in cleaned_texts]

# Assuming 'tokenized_texts' is your list of tokenized tweets
all_words = [word for text in tokenized_texts for word in text]
word_freq = Counter(all_words)
print(word_freq.most_common(20))

[('ubisoft', 107584), ('I', 57887), ('game', 44403), ('creed', 17293), ('assassin', 17045), ('get', 14713), ('thank', 13330), ('play', 11542), ('de', 11434), ('please', 10896), ('new', 10677), ('like', 10388), ('make', 9886), ('one', 8661), ('go', 8184), ('good', 7769), ('issue', 7493), ('far', 6816), ('look', 6751), ('xbox', 6722)]


In [24]:
from gensim.models import CoherenceModel
custom_stopwords = {'ubisoft', 'game', 'go', 'get', 'I'}

stop_words = set(stopwords.words('english'))
stop_words.update(custom_stopwords)



# Create Dictionary and Corpus
dictionary = corpora.Dictionary(tokenized_texts)
dictionary.filter_extremes(no_below=15, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

# Build LDA Model
max_topics = 100
step = 10
for num_topics in range(10, max_topics+step, step):
    print(f"Number of Topics: {num_topics}")
    num_topics = 10
    passes = 15
    random_state = 42

    lda_model = models.LdaMulticore(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        random_state=random_state,
        chunksize=100,
        passes=passes,
        workers=4,
        per_word_topics=True
    )
    
    

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    print(f'Coherence Score: {coherence_score}')
    
    perplexity = lda_model.log_perplexity(corpus)
    print(f'Perplexity: {perplexity}')

    # Print Topics
    for idx, topic in lda_model.print_topics(-1):
        print(f"Topic: {idx}\nWords: {topic}\n")

    # Visualize Topics
    lda_vis = gensimvis.prepare(lda_model, corpus, dictionary)
    # Uncomment the line below to display the visualization in the notebook
    # pyLDAvis.display(data=lda_vis)
    pyLDAvis.save_html(lda_vis, f'lda_visualization_topic_{num_topics}.html')


Number of Topics: 10
Coherence Score: 0.35409867276042295
Perplexity: -7.631308156811642
Topic: 0
Words: 0.050*"game" + 0.042*"make" + 0.023*"hey" + 0.020*"try" + 0.017*"team" + 0.014*"time" + 0.014*"well" + 0.013*"think" + 0.013*"support" + 0.013*"back"

Topic: 1
Words: 0.040*"siege" + 0.039*"pc" + 0.033*"rainbow" + 0.029*"six" + 0.028*"free" + 0.026*"hope" + 0.023*"win" + 0.021*"division" + 0.021*"console" + 0.019*"since"

Topic: 2
Words: 0.139*"creed" + 0.138*"assassin" + 0.031*"give" + 0.026*"guy" + 0.018*"long" + 0.017*"unity" + 0.016*"series" + 0.013*"driver" + 0.012*"anything" + 0.012*"reveal"

Topic: 3
Words: 0.154*"thank" + 0.051*"de" + 0.037*"fix" + 0.028*"open" + 0.020*"gameplay" + 0.018*"la" + 0.018*"en" + 0.018*"que" + 0.015*"red" + 0.015*"th"

Topic: 4
Words: 0.035*"please" + 0.025*"issue" + 0.024*"gt" + 0.017*"still" + 0.017*"work" + 0.017*"sorry" + 0.014*"get" + 0.014*"know" + 0.014*"take" + 0.014*"let"

Topic: 5
Words: 0.057*"update" + 0.049*"use" + 0.043*"beta" + 0.03

KeyboardInterrupt: 

# Experiment with Different Models

In [10]:
import openai
from bertopic import BERTopic
from bertopic.representation import OpenAI

OPEN_AI_API_KEY = "your_openai_api_key"
client = openai.OpenAI(api_key=OPEN_AI_API_KEY)
representation_model = OpenAI(client, model="gpt-4o-mini", chat=True)
topic_model = BERTopic(representation_model=representation_model,language="multilingual")

topics, probabilities = topic_model.fit_transform(texts)

# Explore topics
topic_model.get_topic_info()

c:\Users\Admin\anaconda3\envs\is434\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\anaconda3\envs\is434\Lib\site-packages\umap\__init__.py:36: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
c:\Users\Admin\anaconda3\envs\is434\Lib\site-packages\pkg_resources\__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,62770,-1_they_but_it_like,"[they, but, it, like, them, their, and, dont, ...","[Just finished Assassin's Creed, and all I can..."
1,0,3214,0_rainbow6uk_titaniumrolo_ubisoftuk_lol,"[rainbow6uk, titaniumrolo, ubisoftuk, lol, pre...",[@TitaniumRolo @Rainbow6_UK @Rainbow6Game @Ubi...
2,1,1370,1_assassinsfr_assassinscreed_assassinsuk_ubiso...,"[assassinsfr, assassinscreed, assassinsuk, ubi...",[@CreedKells @Ubisoft @UbisoftDE @UbisoftFR @U...
3,2,910,2_servers_server_down_fix,"[servers, server, down, fix, ubisoftsupport, u...","[@Ubisoft servers down?, @Ubisoft are the serv..."
4,3,905,3_ac_ac3_ac4_ac2,"[ac, ac3, ac4, ac2, ac1, acu, origins, acs, ac...","[@Ubisoft @assassinscreed Best AC game, @Ubiso..."
...,...,...,...,...,...
2261,2260,10,2260_varsitygamingtv_crackheaded_httpstcofwbpz...,"[varsitygamingtv, crackheaded, httpstcofwbpz2j...",[@VarsityGamingTV Come on. You knew this would...
2262,2261,10,2261_fc5_mzammadkhan_chiranramgobin_thatsgroov...,"[fc5, mzammadkhan, chiranramgobin, thatsgroovy...",[@RustyPennyy What platform do you play FC5 on...
2263,2262,10,2262_promises_modern_return_httptcow6slz9vzw0,"[promises, modern, return, httptcow6slz9vzw0, ...",[Ubisoft Promises Modern Day Story Will Return...
2264,2263,10,2263_httptinyurlcombu64oy_httptcoey34fpn5_http...,"[httptinyurlcombu64oy, httptcoey34fpn5, httpbi...",[@FalseShepard we're working on getting more @...


In [ ]:
topic_model.visualize_topics()